### Assignment 5

![image info](https://www.mdpi.com/sensors/sensors-17-01951/article_deploy/html/images/sensors-17-01951-g003.png)


>SqueezeNet: AlexNet-level accuracy with 50x fewer parameters and <0.5MB model size
(https://arxiv.org/abs/1602.07360)

In this assigment you're going to use the pretrained network SqueezeNetv1.2 (~ 5 Mb) 

#### Task 1 (5 points):

go to https://github.com/miaow1988/SqueezeNet_v1.2 and download the 'symbol.json' and '.params' files (there is not a 'synset.txt' file! so don't use these lines, Hint: just comment these lines).

* Install MXNet v1.5 (hint: create a new conda environmet with python 3, pip install mxnet==1.5.1) and follow the same steps of the lecture (part: *Using pre-trained models as feature extractors*). Find the flatten output layer and create a feature extractor (hint: It should be a numpy array of 1000 elements).
* Download the dogs versus cats *training folder* from https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data (Remember the number of images is 12500 for each class).
* Extract the array of features for different number of images (N: 10, 100, 500, 1000, also 5000 and 12500) and for each value train your favorite binary classifier (only one!!!) using GridSearch to optimize some hyperparameters. Consider to use https://notebooks.csc.fi if you have computational limitations. 

* Report the accuracy for each value of N and the computational time during the training step.

#### Task 2 (5 points):

Repeat all previous steps using MobileNet V2 (https://github.com/KeyKy/mobilenet-mxnet). How the two networks compare in terms of accuracy and running time?

**WARNING**: At least for N= 5000 and 12500 it can take some time in your computer, depending of your resources. The time can largely increases depending of your chosen classifier.

#### Task 3 (5 points):

Using the best network. Train a machine learning model able to predic COVID-19 from chest X-Ray images. Use the data from https://www.kaggle.com/praveengovi/coronahack-chest-xraydataset. 

Present and discuss your best model.


# TASK3:

#we will use SqueezeNet model for this task as this is identified as better model among TASK1 & TASK2.
================================================================

In [1]:
#! pip install mxnet-cu101==1.5 --user

     |█████████████████████████▏      | 433.2 MB 34.6      |████████████████████████████████| 551.3 MB 2.7 kB/s  eta 0:00:0104�██▍      | 438.0 MB 34.6 MB/s eta 0:00:04��███████▌      | 439.7 MB 34.6 MB/s eta 0:00:04█████████████▋      | 441.7 MB 34.6 MB/s eta 0:00:04��███████████████████████▊      | 443.5 MB 34.6 MB/s eta 0:00:04��███████▉      | 444.8 MB 34.6 MB/s eta 0:00:04�███      | 446.3 MB 34.6 MB/s eta 0:00:04��████████      | 448.1 MB 34.6 MB/s eta 0:00:03██████████████▏     | 450.0 MB 34.6 MB/s eta 0:00:03��██▎     | 452.1 MB 34.6 MB/s eta 0:00:03��██▍     | 453.7 MB 34.6 MB/s eta 0:00:03��██▍     | 455.0 MB 34.6 MB/s eta 0:00:03��██▋     | 458.8 MB 35.5 MB/s eta 0:00:03MB/s eta 0:00:03| 462.3 MB 35.5 MB/s eta 0:00:03MB/s eta 0:00:03█████████████████████████     | 466.0 MB 35.5 MB/s eta 0:00:03467.6 MB 35.5 MB/s eta 0:00:03��█████████████▎    | 469.4 MB 35.5 MB/s eta 0:00:03��███████████████████████▍    | 471.2 MB 35.5 MB/s eta 0:00:03███████████████████████████▋    | 474.8 

In [2]:
#! pip install mxnet==1.5.1 --user
#! pip install requests matplotlib opencv-python-headless

     |████████████████████████████████| 23.1 MB 2.8 MB/s eta 0:00:01 MB 2.8 MB/s eta 0:00:07B/s eta 0:00:07eta 0:00:06        | 9.4 MB 2.8 MB/s eta 0:00:05█▎                | 11.0 MB 2.8 MB/s eta 0:00:0512.8 MB 2.8 MB/s eta 0:00:04| 14.9 MB 2.8 MB/s eta 0:00:03MB/s eta 0:00:02███████████████████████▎  | 21.2 MB 2.8 MB/s eta 0:00:01��███████████████▉| 23.0 MB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 47.6 MB 55.5 MB/s eta 0:00:01             | 1.1 MB 3.9 MB/s eta 0:00:128 MB 3.9 MB/s eta 0:00:1212                        | 6.2 MB 3.9 MB/s eta 0:00:11 |█████▋                          | 8.3 MB 3.9 MB/s eta 0:00:11 10.3 MB 3.9 MB/s eta 0:00:10                     | 11.9 MB 3.9 MB/s eta 0:00:103.9 MB/s eta 0:00:09��████▉                     | 16.2 MB 3.9 MB/s eta 0:00:09MB/s eta 0:00:08 MB 3.9 MB/s eta 0:00:0807�██████████▊                | 23.3 MB 3.9 MB/s eta 0:00:07��█▍              | 25.8 MB 3.9 MB/s eta 0:00:06��██████████████▋             | 27.8 MB 3.9 MB/s eta 0:00:

In [1]:
import mxnet as mx
path='https://github.com/miaow1988/SqueezeNet_v1.2/'
[mx.test_utils.download(path+'raw/master/model-symbol.json'),
 mx.test_utils.download(path+'raw/master/model-0000.params')]
# mx.test_utils.download(path+'synset.txt')

['model-symbol.json', 'model-0000.params']

In [2]:
sym, arg_params, aux_params = mx.model.load_checkpoint("./model", 0)

In [3]:
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
          label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)
with open('model-symbol.json', 'r') as f:
    labels = [l.rstrip() for l in f]

In [4]:
#! pip install opendatasets

     |████████████████████████████████| 97 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 5.3 MB/s  eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=a4820f78a0bac75966a091a1275aeaa9acee1a316ac81f63b1745f014bd487b3
  Stored in directory: /home/jovyan/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle


In [5]:
#! pip install opendatasets
import opendatasets as od
#import zipfile as zf
od.download("https://www.kaggle.com/praveengovi/coronahack-chest-xraydataset", force=True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: abhimolaj
Your Kaggle Key: ········


  0%|          | 0.00/1.19G [00:00<?, ?B/s]

100%|██████████| 1.19G/1.19G [00:32<00:00, 39.1MB/s]


In [6]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import os

metadata =  pd.read_csv("coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv")
mypath = join(os.getcwd(),'coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train')
all_imgs = [join(mypath,f) for f in listdir(mypath)]

In [7]:
covid_imgs = metadata["X_ray_image_name"][metadata["Label_2_Virus_category"] == "COVID-19"]
no_covid_imgs = metadata["X_ray_image_name"][metadata["Label_2_Virus_category"] != "COVID-19"]
print("covid-19: {} and no-covid-19: {}".format(len(covid_imgs),len(no_covid_imgs)))

covid-19: 58 and no-covid-19: 5852


In [8]:
covid_imgs = [join(mypath,f) for f in covid_imgs ]
no_covid_imgs = [join(mypath,f) for f in no_covid_imgs ]

In [9]:
# list the last 10 layers
all_layers = sym.get_internals()
all_layers.list_outputs()[-10:]

['fire9_concat_output',
 'dropout0_output',
 'conv10_conv_weight',
 'conv10_conv_bias',
 'conv10_conv_output',
 'conv10_relu_output',
 'pool10_output',
 'flatten0_output',
 'softmax_label',
 'softmax_output']

In [10]:
#Define a simple batch
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
# define a simple data batch
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

In [11]:
def get_image(url, show=False):
    if url.startswith('http'):
        # download and show the image
        fname = mx.test_utils.download(url)
    else:
        fname = url
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    if img is None:
         return None
    if show:
         plt.imshow(img)
         plt.axis('off')
    # convert into format (batch, RGB, width, height)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

def predict(url):
    img = get_image(url, show=True)
    # compute the predict probabilities
    mod.forward(Batch([mx.nd.array(img)]))
    prob = mod.get_outputs()[0].asnumpy()
    # print the top-5
    prob = np.squeeze(prob)
    a = np.argsort(prob)[::-1]
    for i in a[0:5]:
        print('probability=%f, class=%s' %(prob[i], labels[i]))


In [12]:
fe_sym = all_layers['flatten0_output']
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

In [13]:
def get_features(img):
    fe_mod.forward(Batch([mx.nd.array(img)]))
    features = fe_mod.get_outputs()[0].asnumpy()
    return features

In [55]:
Nmax = 10
covid_features = [get_features(get_image(img)).ravel() for img in covid_imgs[:Nmax]]
no_covid_features = [get_features(get_image(img)).ravel() for img in no_covid_imgs[:Nmax]]

In [56]:
Y_covid = np.array(Nmax * [1])
Y_no_covid = np.array(Nmax * [0])

In [57]:
X_cvd = np.vstack([covid_features,no_covid_features])
Y_cvd = np.vstack([Y_covid,Y_no_covid]).ravel()

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from time import time
import warnings

param_grid = {'C': [0.001,0.01, 0.1, 1, 3, 5, 7, 10],
              'gamma': [0.001, 0.01, 0.1, 1, 3, 5, 7, 10]}
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=42)

start = time()
grid_search.fit(X_train, y_train)
#report(grid_search.cv_results_)
stop = time()

In [69]:
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
print("gridSearchCV took %.2f seconds" % ((stop - start)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}\n".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

Test set score: 0.80
gridSearchCV took 0.72 seconds
Best parameters: {'C': 1, 'gamma': 0.001}
Best cross-validation score: 0.87

Best estimator:
SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [ ]:
As we see above results using SqueezeNet mxnet model and using gridsearchSVC for calculation of results, 
(i) the test score is 0.80  with processing time 0.60 seconds
(ii) the Best cross validation score is 0.87
We used Nmax=10

When we see the data records, there are 58 covid images against 5852 no-covid images: The results are pretty good. 

And hence we conclude, this model is performing well for covid dataset used.